In [1]:
# Import Libraries

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle
from sklearn.preprocessing import OneHotEncoder

In [2]:
# Load Dataset

data = pd.read_csv('Churn_Modelling.csv')
data.head(2)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [3]:
# Pre-Process the data

## Drop irrelevant features
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1)
data.head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [4]:
## Encode categorical variables

### Encode Gender
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head(6)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
5,645,Spain,1,44,8,113755.78,2,1,0,149756.71,1


In [5]:
### OneHot Encode Geography
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns = onehot_encoder_geo.get_feature_names_out(['Geography'])) # Create dataframe from the OHE operation
print(f"{onehot_encoder_geo.get_feature_names_out(['Geography'])}\n\n{geo_encoded_df.head(7)}")

['Geography_France' 'Geography_Germany' 'Geography_Spain']

   Geography_France  Geography_Germany  Geography_Spain
0               1.0                0.0              0.0
1               0.0                0.0              1.0
2               1.0                0.0              0.0
3               1.0                0.0              0.0
4               0.0                0.0              1.0
5               0.0                0.0              1.0
6               1.0                0.0              0.0


In [6]:
### Combine OHE columns with original data
data = pd.concat([data.drop('Geography', axis = 1),geo_encoded_df], axis = 1)
data.head(6)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
5,645,1,44,8,113755.78,2,1,0,149756.71,1,0.0,0.0,1.0


In [7]:
## Save the encoders and scaler
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [8]:
# Extract dependent and independent features

X = data.drop('Exited', axis = 1)
y = data['Exited']

In [9]:
# Split into test and training set

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 43)

In [10]:
# Scale features

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler,file)

## ANN Implementation

In [12]:
# import libraries

import tensorflow as tf
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [13]:
# Build ANN Model
model = Sequential([
    Dense(64, activation = 'relu', input_shape = (X_train.shape[1],)), ## HL1 connected with input layer
    Dense(32, activation = 'relu'), ## HL2
    Dense(1, activation = 'sigmoid') ## output layer
])


model.summary()

c:\Users\olleyi01\Downloads\Data Science Re-Learning\venv\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Compile  model

opt = tensorflow.keras.optimizers.Adam(learning_rate = 0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

model.compile(optimizer = opt, loss = loss, metrics = ['accuracy'])

In [15]:
## Set up the Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%h%M%S")
log_file_name = datetime.datetime.now().strftime("%Y%m%d-%h%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq = 1)

In [16]:
## Set up Early stopping
early_stopping_callback = EarlyStopping(monitor = 'val_loss',
                                        patience = 15,
                                        restore_best_weights = True)

In [17]:
## Train Model
history = model.fit(
    X_train, y_train, validation_data = (X_test, y_test),
    epochs = 100,
    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8388 - loss: 0.3934 - val_accuracy: 0.8520 - val_loss: 0.3658
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8529 - loss: 0.3557 - val_accuracy: 0.8540 - val_loss: 0.3698
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8589 - loss: 0.3476 - val_accuracy: 0.8475 - val_loss: 0.3642
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8591 - loss: 0.3439 - val_accuracy: 0.8595 - val_loss: 0.3570
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8559 - loss: 0.3429 - val_accuracy: 0.8590 - val_loss: 0.3501
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8637 - loss: 0.3369 - val_accuracy: 0.8585 - val_loss: 0.3609
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8624 - loss: 0.3315 - val_accuracy: 0.8565 - val_loss: 0.3545
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8640 - loss: 0.3327 - val_accu

In [18]:
## Save model
model.save('model.h5')

In [19]:
## Load Tensorboard Extension
%load_ext tensorboard

In [24]:
log_file_name

'20260129-Jan2057'

In [23]:
%tensorboard --logdir logs/fit/$log_file_name

Reusing TensorBoard on port 6006 (pid 29396), started 0:00:07 ago. (Use '!kill 29396' to kill it.)